In [1]:
import time
import secretflow as sf
import spu
import os

start = time.perf_counter()

network_conf = {
    "parties": {
        "alice": {
            "address": "alice:8000",
        },
        "bob": {
            "address": "bob:8000",
        },
    },
}

party = os.getenv("SELF_PARTY", "alice")
sf.shutdown()
sf.init(
    address="127.0.0.1:6379",
    cluster_config={**network_conf, "self_party": party},
    log_to_driver=True,
)

alice's Output:

2024-07-31 13:51:29,315	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.19.0.3:6379...
2024-07-31 13:51:29,377	INFO worker.py:1724 -- Connected to Ray cluster.
2024-07-31 13:51:29.398 INFO api.py:233 [alice] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'alice:8000', 'bob': 'bob:8000'}, 'CURRENT_PARTY_NAME': 'alice', 'TLS_CONFIG': {}}
2024-07-31 13:51:30.023 INFO barriers.py:284 [alice] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=2764) 2024-07-31 13:51:30.019 INFO grpc_proxy.py:359 [alice] -- [Anonymous_job] ReceiverProxy binding port 8000, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCod

bob's Output:

2024-07-31 13:51:29,316	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: 172.19.0.2:6379...
2024-07-31 13:51:29,377	INFO worker.py:1724 -- Connected to Ray cluster.
2024-07-31 13:51:29.401 INFO api.py:233 [bob] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': 'alice:8000', 'bob': 'bob:8000'}, 'CURRENT_PARTY_NAME': 'bob', 'TLS_CONFIG': {}}
2024-07-31 13:51:30.023 INFO barriers.py:284 [bob] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=2494) 2024-07-31 13:51:30.019 INFO grpc_proxy.py:359 [bob] -- [Anonymous_job] ReceiverProxy binding port 8000, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"methodConfig": [{"name": [{"service": "GrpcService"}], "retryPolicy": {"maxAttempts": 5, "initialBackoff": "5s", "maxBackoff": "30s", "backoffMultiplier": 2, "retryableStatusCodes": ["U

In [2]:
alice, bob=sf.PYU("alice"), sf.PYU("bob")
current_dir = os.getcwd()

In [3]:
spu_conf = {
    "nodes": [
        {
            "party": "alice",
            "address": "alice:8001",
            "listen_addr": "alice:8001",
        },
        {
            "party": "bob",
            "address": "bob:8001",
            "listen_addr": "bob:8001",
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}
spu_device=sf.SPU(cluster_def=spu_conf)

In [4]:
report = spu_device.pir_query(
    server="alice",
    client="bob",
    server_setup_path=f"{current_dir}/pir_server_setup",
    client_key_columns=["uid"],
    client_input_path=f"{current_dir}/pir_query.csv",
    client_output_path=f"{current_dir}/pir_result.csv"
)

In [5]:
report

alice's Output:

[{'party': 'alice', 'data_count': 1}, {'party': 'bob', 'data_count': 1}]

bob's Output:

[{'party': 'alice', 'data_count': 1}, {'party': 'bob', 'data_count': 1}]

In [7]:
!cat /root/workspace/pir_result.csv
# 结果为空

uid,ascii_free_email


In [9]:
!cat /tmp/server_secret_key.bin | wc -m

24


In [10]:
!ls -R /root/workspace/pir_server_setup

/root/workspace/pir_server_setup:
bucket_0  pir_meta_info

/root/workspace/pir_server_setup/bucket_0:
bundle_0  db_meta_info

/root/workspace/pir_server_setup/bucket_0/bundle_0:
000005.ldb  000021.log	LOCK  LOG.old
000006.ldb  CURRENT	LOG   MANIFEST-000020

/root/workspace/pir_server_setup/bucket_0/db_meta_info:
000005.ldb  000020.log	CURRENT  LOCK  LOG  LOG.old  MANIFEST-000019

/root/workspace/pir_server_setup/pir_meta_info:
000005.ldb  000022.log	CURRENT  LOCK  LOG  LOG.old  MANIFEST-000021


In [ ]:
# alice的加密数据库正常、key.bin正常
# bob的csv文件正常
# 查询结果为空